# Mock Case Study #001: Malicious File Upload and Web Shell Planting Attack

---

## Incident Description

- Security operations center (SOC) receives an alert from Web Application Firewall (WAF) indicating suspicious file uploads to internal CRM web application.
- Uploaded file named `invoice_check.php` by external IP address.
- Web server responds with HTTP 200 OK status after upload.
- Proxy logs show the same external IP accessing `invoice_check.php` immediately afterward.
(Note: Proxy logs refer to activity captured by the organization's Secure Web Gateway — a modern cloud-based web filtering, threat scanning, and Data Loss Prevention (DLP) enforcement platform, replacing legacy web proxies.)
- No immediate crash or downtime reported, but network team notices unusual outbound traffic from the CRM server.
This outbound traffic anomaly is a critical alert: it suggests the possibility that the CRM server has been compromised and is now communicating externally under attacker control. Potential threats include active Command-and-Control (C2) channel establishment for remote manipulation, as well as unauthorized data exfiltration attempts. Immediate triage and containment are prioritized to prevent further breach progression."
---

## System Anatomy Involved

- **Web Application Layer:** CRM frontend accepting file uploads.
- **Web Server Layer:** Apache HTTP Server hosting the CRM app.
- **Operating System Layer:** Linux (Ubuntu 20.04) hosting the web server.
- **Memory Layer:** Potential runtime execution of unauthorized code (web shell in RAM).
"Given the evidence suggesting a possible active web shell (an unauthorized script running within server memory), immediate isolation of the CRM server is the first priority to prevent further attacker command-and-control (C2) activity or data exfiltration. However, while memory analysis becomes critically important, it should follow structured triage order: after initial application log review and host/network cross-reference are completed to build a clear timeline and validate the compromise. Memory capture and forensic inspection are prioritized if live compromise indicators escalate during early triage stages. A trusted forensic RAM capture tool would be used post-isolation to preserve volatile evidence, such as the running web shell process, injected code fragments, or in-memory payloads before any system reboot or alteration.

## Initial Triage Framework Selection

**Primary Trigger Type:**  
➔ Application-Based Triage Framework

**Reason:**  
- Suspicious file upload to application layer.
- Follow-up abnormal server activity suggesting possible exploitation.

---

## Step-by-Step Triage Execution

### 1. Application Log Review

- Reviewed Apache server access logs:
  - Found POST request from `203.0.113.25` uploading `invoice_check.php`.
  A POST request is an HTTP method where a client (like a browser or attacker) submits data to a server — typically used for uploading files, submitting forms, or sending API payloads.
  - Found subsequent GET request to access `invoice_check.php`.
  A .php file contains code written in PHP (Hypertext Preprocessor), a widely used server-side scripting language for creating dynamic web content — and crucially, PHP code is executed on the server."
  The attacker first used a POST request to upload the malicious file (invoicecheck.php) to the server, then immediately sent a GET request to access and trigger the execution of the uploaded file, indicating active exploitation. Proper file upload validation is considered a baseline web security control today; lack of validation is a critical vulnerability (and appears on OWASP Top 10 risks)."
- CRM application logs showed no internal validation on file type or file content at upload.
"Because PHP files are executed on the server side, any malicious code they contain runs with the server’s privileges, not just in the attacker’s browser — making server-side execution far more dangerous than simple client-side scripts like JavaScript."
Internal validation refers to the application's ability to inspect uploaded files and reject those that are dangerous, like executable scripts. In this case, the CRM accepted any file upload without checking MIME types, extensions, or file contents — a serious vulnerability because it allowed the upload of executable PHP code instead of safe document files like PDFs or images.
MIME (Multipurpose Internet Mail Extensions) types identify the nature of a file — such as application/pdf for a PDF or image/png for an image — and proper upload filters should check that uploaded files match allowed MIME types to block executable content."
S/MIME (Secure/Multipurpose Internet Mail Extensions) is a cryptographic protocol for email encryption and signing, unrelated to basic MIME type checking at upload — here, we are referring only to MIME file type headers
Proper file upload validation is considered a baseline web security control today; lack of validation is a critical vulnerability (and appears on OWASP Top 10 risks). File extension validation checks the filename (e.g., .jpg, .docx) to restrict allowed uploads, but extension checks alone are weak; robust systems inspect the actual file content to detect disguised executables hidden behind harmless-looking names.
In secure systems, proper file upload validation checks both the declared MIME type and the real file content, not just the filename extension.



### 2. Cross-Reference Host and Network Activity

- Host Security Event Logs:
Host security event logs:
On Linux servers: These are typically found in system directories like /var/log/auth.log, /var/log/secure, or /var/log/syslog, and they capture user authentication attempts, process creation events, and security-related system messages.
On Windows servers: Security event logs are accessible via the Windows Event Viewer (Security section) and record user logins, privilege use, process creation, and sensitive system events.
Together, they serve as the local audit trail for detecting suspicious host-level behavior.

  - Detected new unexpected process spawn tied to Apache web server user.
  - Parent: `apache2`, Child: `/bin/bash`Bash (/bin/bash) is the standard Linux command-line shell, allowing direct command execution; if spawned unexpectedly by a web server, it often signals remote attacker access or script execution attempts.
  Unexpected process spawn:
A "spawn" refers to a new child process being created by an existing parent process. An unexpected process spawn means a process launch that is atypical for the parent application’s normal operation — in this case, an unusual shell (/bin/bash) created by the Apache web server (apache2), suggesting remote code execution or web shell activity.

- NetFlow Analysis:
  - Outbound connections from CRM server to unknown external IPs over high ports (TCP 54545, 58332).
  High ports (e.g., TCP 54545, 58332) significance:
High-numbered TCP ports (above 49152) are typically used for dynamic or ephemeral outbound connections.
Dynamic or ephemeral outbound connections:
These are short-lived TCP/UDP ports automatically assigned by the operating system for outbound communication, typically in the high port range (49152–65535), used for temporary web browsing, API calls, or short client sessions.
Their presence from a server that usually serves inbound web traffic only is suspicious.
Attackers often use high ports to blend outbound C2 (Command and Control) or exfiltration traffic into normal network noise.
Legitimate enterprise services usually communicate over well-known ports (80, 443, 22, etc.), so arbitrary high port outbound connections are a strong indicator of compromise.
  
  - After-hours activity (server had no reason to initiate external connections).
  Why After Hours Outbound Activity Matters:
Business servers like CRM applications should generally not initiate random outbound connections, especially after normal operating hours.
Outbound traffic from a server that should be mostly inbound-facing suggests possible attacker control, data exfiltration, or external C2 beaconing attempts.
Why attacker blending didn’t work here:
Although the attackers used high ephemeral ports to mask their C2 or exfiltration attempts among normal traffic, the CRM server’s baseline behavior typically involved only inbound client connections, not spontaneous outbound traffic, especially after hours. This unusual outbound behavior still stood out when baseline server behavior was reviewed.

Time-of-day anomalies strengthen the overall suspicion that the server is actively compromised.

Whenever unexpected outbound traffic from a server is detected — particularly after business hours — investigation immediately prioritizes:
Attacker command/control (C2): Is the server remotely controlled?
Data exfiltration: Is sensitive data leaking out?
Beaconing: Is the server periodically signaling an external attacker awaiting further instructions?



### 3. Authentication and Identity Validation

- No abnormal CRM user authentication detected.
- No legitimate CRM users performed file upload — indicates external exploitation, not insider threat.
Where CRM user authentication data is found:
CRM user authentication events are typically logged within the CRM application’s own audit logs or access logs, which track login attempts, successful authentications, failed logins, and session creation metadata. Larger SaaS CRM systems (like Salesforce) may expose these authentication logs via an administrative dashboard or API access.
If the CRM was self-hosted (e.g., an internal CRM built on LAMP stack or Django), then user authentication logs might also partially exist in the web server logs (e.g., Apache or NGINX access logs) alongside the application’s own logging system.


### 4. Integrity Validation

- File system inspection:
  - Found `invoice_check.php` stored under `/var/www/html/uploads/`.
- Hash comparison:
  - `invoice_check.php` hash not matching any known internal deployment files.
- Application config review:
  - File upload validation improperly configured (allowed arbitrary PHP files).

### 5. Memory and Runtime Inspection

- RAM capture taken using `volatility3` framework.
- Process list revealed active `bash` session under Apache process space.
- Memory strings showed presence of command-and-control (C2) beacon scripts hidden inside `/tmp/` folder.

What is Volatility-3 Framework:
Volatility-3 is an advanced open-source memory forensic framework designed to analyze RAM captures for evidence of malware, active processes, injected code, credential theft artifacts, and hidden attacker activity. It supports modern Windows, Linux, and macOS systems, and allows investigators to inspect memory images without relying on the compromised operating system.

Additional Memory and Runtime Inspection Expansion:

RAM Capture Purpose: Memory captures preserve volatile evidence — including live malware, decrypted payloads, shell sessions, and active network connections — that would otherwise be erased on reboot or shutdown.

Process List Analysis: Inspecting active processes helps detect unexpected shells, malware executables, or hijacked system binaries running covertly.

Memory String Analysis: Extracting readable text (strings) from RAM can reveal attacker command-and-control scripts, stolen credential traces, or encrypted payloads loaded only in memory (not on disk).

Common Artifacts: Attackers often stash temporary tools, payloads, or beacon scripts inside folders like /tmp/ on Linux systems because /tmp is writable and often overlooked.

Smooth Insert for your case study (after your existing points):

"Using Volatility-3, memory forensics revealed live evidence of attacker persistence — specifically an unauthorized bash shell session running inside the web server’s memory space, and hidden command-and-control beacon scripts in the temporary filesystem, highlighting active exploitation."

Summary View for Learning Reinforcement:


Concept	Quick Explanation
Volatility-3	Memory forensic tool for analyzing RAM captures
RAM Capture	Preserves live malware, processes, decrypted artifacts
Process List	Identify unauthorized shells or hijacked binaries
Memory Strings	Extract readable clues like scripts, creds, payloads
/tmp/ Folder	Common attacker hiding spot for tools/scripts


## Root Cause Analysis

- CRM application failed to sanitize and validate uploaded files properly.
- Attacker planted a web shell (PHP script) allowing remote command execution.
- Web server executed attacker’s code with web server privileges (`www-data` user). Since the PHP shell was executed via the Apache web server process, the attacker's code inherited the server process's system-level privileges (e.g., running as the www-data user on Linux), enabling file read/write within allowed directories.
Outbound C2 connections indicated active remote control. However, no direct evidence of lateral movement within the internal network was confirmed from initial triage."


## Containment Actions

- Immediate isolation of the CRM server from the network (switch-level port shutdown).
- Backup application data for forensic analysis.
- Eradicated malicious PHP file and hidden `/tmp/` artifacts. Eradicated malicious PHP file from the application uploads directory and hidden script artifacts from the server's /tmp filesystem directory. Memory-resident artifacts cleared during server isolation and reboot.
- Rebuilt CRM server from known good backup after full forensic review.
- Patched CRM app to enforce strict file type validation (whitelisting only safe file extensions like `.jpg`, `.pdf`).

---

## Lessons Learned

- Always enforce strict file validation in web applications, not just extension checks but MIME-type and content inspection.
- Web servers must run with minimum required privileges.
- Regular web application penetration testing could have uncovered this vulnerability earlier.
- Memory forensics critical when RAM-resident malware and shells are suspected.

This mock case study of a malicious file upload and web shell planting attack illustrates the anatomy of a modern application-layer intrusion and its full lifecycle across investigation, containment, and remediation.
It highlights several critical real-world principles:
Weak application-layer controls — like poor file upload validation — are still primary footholds for attackers.
Server-layer compromise often stems from overlooked permissions and lack of application hardening.
Memory and runtime forensics become essential when active exploitation is suspected, not just disk-based analysis.
Cross-referencing host, network, and application layers builds a complete, defensible investigation timeline.
Early detection through proper logging, WAF alerts, and triage frameworks dramatically reduces attacker dwell time.
This project demonstrates a practical example of how cybersecurity triage frameworks, system anatomy understanding, and disciplined investigation flow allow security professionals to respond systematically — not chaotically — under real-world attack conditions.
The methodology used here deliberately bridges theoretical exam preparation (e.g., CySA+) with real-world investigative logic, preparing practitioners for both technical interviews and operational roles.





